### Importing libraries needed for project

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### Read in our stroke data

In [53]:
stroke_data = pd.read_csv('Desktop/datasets/healthcare-dataset-stroke-data.csv')

In [54]:
pd.set_option('display.max_rows', 100)

In [55]:
stroke_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [56]:
stroke_data['age'].max()

82.0

In [57]:
stroke_data['age'].min()

0.08

### Check rate of hypertension between each gender to see if any is more prone

In [73]:
gender_hypertension_df = stroke_data[['gender','age','stroke']]

In [74]:
gender_hypertension_df

,gender,age,stroke
0,Male,67.0,1
1,Female,61.0,1
2,Male,80.0,1
3,Female,49.0,1
4,Female,79.0,1
...,...,...,...
5105,Female,80.0,0
5106,Female,81.0,0
5107,Female,35.0,0
5108,Male,51.0,0


#### Checking if there a relationship between age and glucose level using a linear model

In [75]:
age = stroke_data[['age']]

In [76]:
average_glucose_level = stroke_data['avg_glucose_level']

In [93]:
print(age.shape)
print(average_glucose_level.shape)

(5110, 1)
(5110,)
